In [5]:
import sys
import os
sys.path.append(os.path.abspath("Algorithms/"))
sys.path.append(os.path.abspath("models/"))

In [6]:
from FBSNNs import *
from BlackScholesBarenblatt100D import *

In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 100  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1.0, 0.5] * int(D / 2))[None, :]
T = 1.0

"Available architectures"
mode = "Naisnet"  # FC and Naisnet are available
activation = "Sine"  # Sine, ReLU and Tanh are available
model = BlackScholesBarenblatt(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 2 * 10**4
lr = 1e-2

In [12]:
tot = time.time()
print(model.device)
graph = model.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 0, Loss: 5.616e+05, Y0: 68.969, Time: 0.42, Learning Rate: 1.000e-02
It: 100, Loss: 2.428e+08, Y0: -81.579, Time: 56.51, Learning Rate: 1.000e-02
It: 200, Loss: 1.129e+07, Y0: 23.472, Time: 57.06, Learning Rate: 1.000e-02
It: 300, Loss: 1.679e+09, Y0: 3.523, Time: 55.99, Learning Rate: 1.000e-02
It: 400, Loss: 1.463e+09, Y0: 20.897, Time: 53.12, Learning Rate: 1.000e-02
It: 500, Loss: 2.062e+09, Y0: -26.567, Time: 51.85, Learning Rate: 1.000e-02
It: 600, Loss: 1.541e+09, Y0: -36.884, Time: 56.60, Learning Rate: 1.000e-02
It: 700, Loss: 1.866e+09, Y0: 17.553, Time: 56.98, Learning Rate: 1.000e-02
It: 800, Loss: 1.250e+09, Y0: 4.257, Time: 55.05, Learning Rate: 1.000e-02
It: 900, Loss: 1.833e+09, Y0: 10.895, Time: 56.82, Learning Rate: 1.000e-02
It: 1000, Loss: 2.656e+09, Y0: -0.538, Time: 56.96, Learning Rate: 1.000e-02
It: 1100, Loss: 2.159e+09, Y0: -28.207, Time: 56.82, Learning Rate: 1.000e-02
NaN encountered in loss at iteration 1113
NaN encountered in loss at iteration 1

KeyboardInterrupt: 

In [5]:
model.load_model("models/BlackScholesBarenblattMLMC100DLBFGS.pth")

In [13]:
graph = model.iteration, model.training_loss

In [14]:
def figsize(scale, nplots = 1):
    fig_width_pt = 438.17227                        
    inches_per_pt = 1.0/72.27                       
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            
    fig_width = fig_width_pt*inches_per_pt*scale    
    fig_height = nplots*fig_width*golden_mean             
    fig_size = [fig_width,fig_height]
    return fig_size

In [15]:
np.random.seed(65)
t_test, W_test = model.fetch_minibatch()
X_pred, Y_pred = model.predict(Xi, t_test, W_test)
samples = 5

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

Y_test = np.reshape(u_exact(T,np.reshape(t_test[0:M, :, :], [-1, 1]), np.reshape(X_pred[0:M, :, :], [-1, D])),
                    [M, -1, 1])

In [18]:
model.save_model("models/BlackScholesBarenblattMLMC100DLBFGS.pth")